<a href="https://colab.research.google.com/github/JSJeong-me/LGE-PRI-1st/blob/main/08-Soldering-Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Annotate the uploaded slide with the given JSON "docstring" regions.
# Strategy:
# 1) Load the slide image.
# 2) Detect the 5 "patch" image regions automatically by finding high-saturation areas (the PCB crops).
# 3) Map JSON entries patch_1..patch_5 to those detected regions in reading order (top-left to bottom-right).
# 4) Convert the normalized solder ROI bboxes to absolute coordinates within each patch and draw them.
# 5) Save an annotated image and the JSON to files for download.

from PIL import Image, ImageDraw, ImageFont
import numpy as np
import json, io, os, textwrap

# -------------- Load image --------------
img_path = "/content/image.png"
im = Image.open(img_path).convert("RGB")
w, h = im.size

# -------------- Detect patch regions --------------
# We'll use HSV saturation to find colorful, high-variance regions (the PCB crops).
import cv2
cv_img = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)
hsv = cv2.cvtColor(cv_img, cv2.COLOR_BGR2HSV)
sat = hsv[:,:,1]
# Threshold on saturation (tune empirically), and clean with morphology
_, sat_mask = cv2.threshold(sat, 35, 255, cv2.THRESH_BINARY)  # 0..255
kernel = np.ones((7,7), np.uint8)
sat_mask = cv2.morphologyEx(sat_mask, cv2.MORPH_OPEN, kernel, iterations=2)
sat_mask = cv2.morphologyEx(sat_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

# Find contours and filter by area & shape
contours, _ = cv2.findContours(sat_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
boxes = []
for cnt in contours:
    x,y,wc,hc = cv2.boundingRect(cnt)
    area = wc*hc
    # Heuristics: big enough, not extreme aspect ratio
    if area > (w*h)*0.02 and 0.3 < wc/max(hc,1) < 3.5:
        # Expand box slightly to include borders
        pad = int(min(w,h)*0.005)
        x0 = max(0, x - pad); y0 = max(0, y - pad)
        x1 = min(w, x+wc + pad); y1 = min(h, y+hc + pad)
        boxes.append((x0,y0,x1,y1))

# If more than 5, take the 5 largest by area
boxes = sorted(boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)[:5]

# Order them in reading order: top-to-bottom, left-to-right
def reading_order_key(b):
    x0,y0,x1,y1 = b
    return (y0//(h/3), x0)
boxes = sorted(boxes, key=reading_order_key)

# -------------- JSON docstring --------------
doc = r'''
{
"metadata": {
"line_id": "L1",
"model": "PCB-Unknown",
"board_rev": "NA",
"video_source": "image://uploaded_slide_page",
"timestamp": "2025-08-18T00:00:00+09:00",
"detector": "VisionTransformer",
"detector_config": "ViTDet-B/16-FPN, ms-test"
},
"summary": { "total_components": 5, "defect_count": 0, "pqi": 100.0 },
"detections": {
"coordinate_type": "normalized_to_patch",
"solder_regions": [
{
"component_id": "patch_1_resistor_h",
"package_est": "06030805",
"frame_idx": 0,
"time_sec": 0.0,
"solder_roi_bboxes_norm": [
[0.18, 0.45, 0.16, 0.28],
[0.66, 0.44, 0.16, 0.28]
],
"conf": 0.86,
"evidence_uri": "annotated://patch_1.png"
},
{
"component_id": "patch_2_resistor_v",
"package_est": "12062010",
"frame_idx": 0,
"time_sec": 0.0,
"solder_roi_bboxes_norm": [
[0.15, 0.08, 0.70, 0.20],
[0.15, 0.78, 0.70, 0.20]
],
"conf": 0.88,
"evidence_uri": "annotated://patch_2.png"
},
{
"component_id": "patch_3_SOT23",
"package_est": "SOT-23",
"frame_idx": 0,
"time_sec": 0.0,
"solder_roi_bboxes_norm": [
[0.27, 0.20, 0.18, 0.26],
[0.71, 0.20, 0.18, 0.26],
[0.44, 0.60, 0.22, 0.26]
],
"conf": 0.83,
"evidence_uri": "annotated://patch_3.png"
},
{
"component_id": "patch_4_resistor_h",
"package_est": "0603~0805",
"frame_idx": 0,
"time_sec": 0.0,
"solder_roi_bboxes_norm": [
[0.16, 0.44, 0.18, 0.30],
[0.68, 0.44, 0.18, 0.30]
],
"conf": 0.87,
"evidence_uri": "annotated://patch_4.png"
},
{
"component_id": "patch_5_shield_can",
"package_est": "Shield/Can",
"frame_idx": 0,
"time_sec": 0.0,
"solder_roi_bboxes_norm": [
[0.05, 0.05, 0.90, 0.08],
[0.05, 0.87, 0.90, 0.08],
[0.05, 0.05, 0.08, 0.90],
[0.87, 0.05, 0.08, 0.90]
],
"note": "실제는 둘레 다수의 소형 조인트; 영상 입력 시 세그멘테이션으로 세부 마스크 제공",
"conf": 0.72,
"evidence_uri": "annotated://patch_5.png"
}
]
},
"defects": []
}
'''
data = json.loads(doc)

solder_regions = data["detections"]["solder_regions"]

# Safety: if detection boxes fewer than 5, pad with repeats
if len(boxes) < len(solder_regions):
    # Just duplicate the last box if needed
    while len(boxes) < len(solder_regions):
        boxes.append(boxes[-1])

# -------------- Draw annotations --------------
draw = ImageDraw.Draw(im, "RGBA")

# Utility to convert normalized bbox (cx,cy,w,h?) or (x,y,w,h)?
# The provided appears to be [x, y, w, h] normalized in [0,1] (top-left based).
def draw_norm_box_on_patch(patch_box, nb, color=(0,255,0,180), width=4):
    x0,y0,x1,y1 = patch_box
    pw = x1 - x0; ph = y1 - y0
    # nb = [x,y,w,h] normalized to the patch
    x, y, bw, bh = nb
    ax0 = int(x0 + x*pw)
    ay0 = int(y0 + y*ph)
    ax1 = int(ax0 + bw*pw)
    ay1 = int(ay0 + bh*ph)
    draw.rectangle([ax0, ay0, ax1, ay1], outline=color, width=width)
    return (ax0, ay0, ax1, ay1)

# Colors for each patch
colors = [(0, 255, 0, 200), (255, 165, 0, 200), (0, 128, 255, 200), (255, 0, 255, 200), (255, 0, 0, 200)]

label_bg = (0,0,0,160)
text_color = (255,255,255,255)

# Try to get a reasonable font
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", size=int(h*0.018))
except:
    font = ImageFont.load_default()

for i, region in enumerate(solder_regions):
    patch_box = boxes[i]
    # Draw patch outline
    draw.rectangle(patch_box, outline=colors[i%len(colors)], width=6)
    # Title on patch
    title = f"{region['component_id']} ({region.get('package_est','')}) | conf {region.get('conf',0):.2f}"
    tx, ty = patch_box[0]+8, patch_box[1]+8
    tw, th = draw.textbbox((0,0), title, font=font)[2:]
    draw.rectangle([tx-4, ty-2, tx+tw+6, ty+th+2], fill=label_bg)
    draw.text((tx, ty), title, font=font, fill=text_color)

    # Draw each solder ROI box
    for nb in region["solder_roi_bboxes_norm"]:
        bx = draw_norm_box_on_patch(patch_box, nb, color=colors[i%len(colors)], width=4)

# Add a side panel with JSON summary (wrapped)
panel_w = int(w*0.36)
panel_x0 = w - panel_w - 20
panel = Image.new("RGBA", (panel_w, h-40), (255,255,255,220))
im.paste(panel, (panel_x0, 20), panel)
panel_draw = ImageDraw.Draw(im, "RGBA")

def wrap_text(text, width_chars=68):
    return "\n".join(textwrap.wrap(text, width=width_chars))

# Compose a compact JSON summary (metadata+summary)
meta = data["metadata"]
summary = data["summary"]
side_text = (
    f"JSON Summary\n"
    f"line_id: {meta['line_id']}   model: {meta['model']}\n"
    f"rev: {meta['board_rev']}    detector: {meta['detector']}\n"
    f"config: {meta['detector_config']}\n"
    f"total_components: {summary['total_components']}  defects: {summary['defect_count']}  PQI: {summary['pqi']}\n\n"
    f"Detections:\n" + "\n".join([
        f"- {r['component_id']} ({r.get('package_est','')}), conf {r.get('conf',0):.2f}, n_roi={len(r['solder_roi_bboxes_norm'])}"
        for r in solder_regions
    ])
)
# Draw the text inside the panel
panel_draw.text((panel_x0+16, 30), side_text, font=font, fill=(0,0,0,255))

# -------------- Save outputs --------------
annotated_path = "/content/annotated_aoi_vit_overlay.png"
im.save(annotated_path)

json_path = "/content/aoi_result_docstring.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

annotated_path, json_path
